## Hierarchical clustering of modalities

In [1]:
import six

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline



import flotilla
flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects'

study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian', flotilla_dir=flotilla_dir)

not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/flotilla/__init__.py:55: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  load_species_data=load_species_data)


2016-02-12 18:23:43	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_bayesian/datapackage.json
2016-02-12 18:23:43	Parsing datapackage to create a Study object
2016-02-12 18:24:30	Initializing Study
2016-02-12 18:24:30	Initializing Predictor configuration manager for Study
2016-02-12 18:24:30	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2016-02-12 18:24:30	Added ExtraTreesClassifier to default predictors
2016-02-12 18:24:30	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2016-02-12 18:24:30	Added ExtraTreesRegressor to default predictors
2016-02-12 18:24:30	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2016-02-12 18:24:30	Added GradientBoostingClassifier to default predictors
2016-02-12 18:24:30	Predictor GradientBoostingRegressor is of type <class 'sklearn.

In [2]:

not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)
splicing_singles_no_outliers = study.splicing.singles.ix[not_outliers]


## Clustering of bimodal events

In [3]:
folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian/modality_clustering/'

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [4]:
sns.set(context='paper', style='ticks')

In [5]:
bimodal = study.supplemental.modalities_tidy.loc[study.supplemental.modalities_tidy.modality == 'bimodal']

for phenotype, df in bimodal.groupby('phenotype'):
#     sample_colors = study.sample_id_to_color[psi_subset.index]
    samples = study.sample_subset_to_sample_ids(phenotype)

    psi_subset = splicing_singles_no_outliers.loc[samples, df.event_name].dropna(how='all', axis=1).dropna(how='all', axis=0)
    mask = psi_subset.isnull()
    corr = psi_subset.corr()
    
    fig, ax = plt.subplots(figsize=(2, 2))
    mean = corr.mean().mean()
    std = corr.std().std()
    cutoff = mean + 3*std
    sns.distplot(corr.values.flat, bins=np.linspace(-1, 1, 50), ax=ax)
    ymin, ymax = ax.get_ylim()
    ax.vlines(cutoff, ymin, ymax, linestyle='--')
    ax.vlines(-cutoff, ymin, ymax, linestyle='--')
    ax.set(title=phenotype)
    
    corr_filtered = corr[corr.abs() >= 0.5]
    
#     g = sns.clustermap(psi_subset.fillna(0.5), mask=mask, metric='euclidean', cmap='RdYlBu_r', center=0.5, 
#     #                    col_cluster=False,
#                        cbar_kws=dict(label='$\Psi$'), method='ward')
#     g.ax_heatmap.set(xticks=[], yticks=[], xlabel='Bimodal splicing events', ylabel='Single {}s'.format(phenotype))
#     g.savefig('{}/bimodal_{}.pdf'.format(folder, phenotype))

AttributeError: 'DataFrame' object has no attribute 'event_name'

In [ ]:
phenotype = 'MN'
df = bimodal.groupby('phenotype').get_group(phenotype)

In [ ]:
samples = study.sample_subset_to_sample_ids(phenotype)

psi_subset = splicing_singles_no_outliers.loc[samples, df.event_name].dropna(how='all', axis=1).dropna(how='all', axis=0)
mask = psi_subset.isnull()
corr = psi_subset.corr()


In [ ]:
corr_subset = corr[corr.abs() >= 0.5]

In [ ]:
corr_subset.shape

In [ ]:
corr_subset.count()

In [ ]:
count = corr_subset.count()
count.mean()

In [ ]:
count.std()

In [ ]:
sns.distplot(count)

In [ ]:
smarce1_events = study.splicing.maybe_renamed_to_feature_id('SMARCE1').intersection(corr_subset.columns)
snap25_events = study.splicing.maybe_renamed_to_feature_id('SNAP25').intersection(corr_subset.columns)

In [ ]:
count[smarce1_events]

In [ ]:

count_cutoff = count.mean() - count.std()
corr_filtered = corr.loc[count > count_cutoff, count > count_cutoff]
corr_filtered

In [ ]:
psi_folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian/permutations/psi'

dfs = []

for csv in sorted(glob.glob('{}/*.csv'.format(psi_folder))):
    psi = pd.read_csv(csv, index_col=0)
    df = psi.groupby(study.sample_id_to_phenotype).apply(
        lambda x: ws.fit_transform(x.dropna(how='all', axis=1)))
    df['iteration'] = int(os.path.basename(csv).split('.')[0].split('_')[1])
    df = df.reset_index()
    dfs.append(df)
    
waypoints = pd.concat(dfs, ignore_index=True)
# sizes.columns = 

In [ ]:
sns.clustermap(corr_filtered.fillna(0), metric='cityblock')

In [ ]:
snap25_smarce1 = corr_subset.loc[smarce1_events, snap25_events]
six.print_(snap25_smarce1.shape)
# sns.heatmap(snap25_smarce1)
snap25_smarce1

In [ ]:
sns.clustermap(corr_subset.fillna(0), method='ward')

In [ ]:
corr_subset.dropna(thresh=1).shape

### Cluster events that are bimodal in all - does it partition the celltypes?

In [ ]:
same_modality_all_celltypes = study.supplemental.modalities_tidy.groupby(['event_name', 'modality']).filter(lambda x: len(x) == 3)
print same_modality_all_celltypes.shape
same_modality_all_celltypes.head()
bimodal_all = same_modality_all_celltypes.event_name[same_modality_all_celltypes.modality == 'bimodal'].unique()
psi_subset = splicing_singles_no_outliers.ix[:, bimodal_all].dropna(thresh=splicing_singles_no_outliers.shape[0]*0.5, axis=1)
print psi_subset.shape

row_colors = study.sample_id_to_color[psi_subset.index]

mask = psi_subset.isnull()
psi_subset = psi_subset.fillna(0.5)
xticklabels = study.splicing.feature_data.gene_name[psi_subset.columns]
psi_ordering_scores = psi_subset.apply(flotilla.compute.splicing.switchy_score)
psi_ordering_scores.sort()

# psi_subset = psi_subset.ix[:, psi_ordering_scores.index]

In [ ]:
same_modality_all_celltypes.groupby(['modality']).size()/3

In [ ]:
folder = '.'

In [ ]:
# # non-outlier single cells and pooled
# sample_ids = psi_filtered.index.tolist() + study.splicing.pooled.index.tolist()

# for (event_name, modality), df in same_modality_all_celltypes.groupby(['event_name', 'modality']):
#     if modality in ('~0', '~1'):
#         continue
    
#     gene_name = study.splicing.feature_data.gene_name[event_name]
#     gene_id = study.expression.data.columns.intersection(study.splicing.feature_data.ensembl_id[event_name].split(','))[0]
#     print gene_name, event_name, modality

#     fig, axes = plt.subplots(figsize=(8, 4), ncols=2)    
#     study.splicing._violinplot(event_name, phenotype_groupby=study.sample_id_to_phenotype,
#                                sample_ids=sample_ids,
#                               phenotype_order=study.phenotype_order, ax=axes[0], color=study.phenotype_color_ordered,
#                               bw=0.2, ylim=(0, 1), yticks=(0, 0.5, 1), ylabel='$\Psi$')
#     study.expression._violinplot(gene_id, phenotype_groupby=study.sample_id_to_phenotype,
#                                sample_ids=sample_ids,
#                               phenotype_order=study.phenotype_order, ax=axes[1], color=study.phenotype_color_ordered,)
#     sns.despine()
#     fig.tight_layout()
#     fig.savefig('{}/same_modality_all_celltypes_{}_{}_{}.pdf'.format(folder, modality, gene_name, event_name))


In [ ]:
sample_colors = study.sample_id_to_color[psi_subset.index]

g = sns.clustermap(psi_subset, mask=mask, metric='euclidean', cmap='RdYlBu_r', center=0.5, 
#                    col_cluster=False,
                   cbar_kws=dict(label='$\Psi$'), method='ward', xticklabels=xticklabels, row_colors=sample_colors)
g.ax_heatmap.set(xticks=[], yticks=[], xlabel='Bimodal splicing events', ylabel='Single cells')
g.savefig('{}/bimodal_all_celltypes_clustergrid.pdf'.format(folder))

g = sns.clustermap(psi_subset.T.corr().dropna(how='all', axis=1).dropna(how='all', axis=0), 
                   metric='euclidean', cmap='PuOr', center=0,  col_colors=sample_colors, row_colors=sample_colors,
                   cbar_kws=dict(label='Pearson $R$'), method='ward')
g.ax_heatmap.set(xlabel='Single cells', ylabel='Single cells')
g.savefig('{}/bimodal_all_celltypes_samples_correlated_clustergrid.pdf'.format(folder))

g = sns.clustermap(psi_subset.corr().dropna(how='all', axis=1).dropna(how='all', axis=0), 
                   metric='euclidean', cmap='PuOr', center=0, 
                   cbar_kws=dict(label='Pearson $R$'), method='ward', xticklabels=xticklabels,
                  yticklabels=xticklabels)
g.ax_heatmap.set(xlabel='Bimodal splicing events', ylabel='Bimodal splicing events', xticks=[], yticks=[])
g.savefig('{}/bimodal_all_celltypes_events_correlated_clustergrid.pdf'.format(folder))

### Bimodal TFs

In [ ]:
bimodal = study.supplemental.modalities_tidy.loc[study.supplemental.modalities_tidy.modality == 'bimodal']
bimodal = bimodal.join(study.splicing.feature_data, on='event_name')
bimodal_tfs = bimodal.loc[bimodal['bioguo_animaltfdb_tf'].fillna(False)]
bimodal_tf_events = bimodal_tfs.event_name.unique()
print bimodal_tf_events.shape

psi_bimodal_tfs = splicing_singles_no_outliers.ix[:, bimodal_tf_events].dropna(
    thresh=splicing_singles_no_outliers.shape[0]*0.5, axis=1)

sample_colors = study.sample_id_to_color[psi_bimodal_tfs.index]

mask = psi_bimodal_tfs.isnull()
psi_bimodal_tfs = psi_bimodal_tfs.fillna(0.5)
xticklabels = study.splicing.feature_data.gene_name[psi_bimodal_tfs.columns]


splicing_label = 'Bimodal splicing events in TFs'

g = sns.clustermap(psi_bimodal_tfs, mask=mask, metric='euclidean', cmap='RdYlBu_r', center=0.5, 
                   cbar_kws=dict(label='$\Psi$'), method='ward', xticklabels=xticklabels, row_colors=sample_colors)
# g.ax_heatmap.set(xticks=[], yticks=[], xlabel=splicing_label, ylabel='Single cells')
g.savefig('{}/bimodal_tfs_celltypes_clustergrid.pdf'.format(folder))

g = sns.clustermap(psi_bimodal_tfs.T.corr().dropna(how='all', axis=1).dropna(how='all', axis=0), 
                   metric='euclidean', cmap='PuOr', center=0, figsize=(20, 20),
                   cbar_kws=dict(label='Pearson $R$'), method='ward', col_colors=sample_colors, row_colors=sample_colors)
g.ax_heatmap.set(xlabel='Single cells', ylabel='Single cells')
g.savefig('{}/bimodal_tfs_celltypes_samples_correlated_clustergrid.pdf'.format(folder))

g = sns.clustermap(psi_bimodal_tfs.corr().dropna(how='all', axis=1).dropna(how='all', axis=0), 
                   metric='euclidean', cmap='PuOr', center=0, 
                   cbar_kws=dict(label='Pearson $R$'), method='ward', xticklabels=xticklabels,
                  yticklabels=xticklabels)
# g.ax_heatmap.set(xlabel=splicing_label, ylabel=splicing_label, xticks=[], yticks=[])
g.savefig('{}/bimodal_tfs_celltypes_events_correlated_clustergrid.pdf'.format(folder))

### Bimodal RBPs which bind mRNA

In [ ]:
bimodal = study.supplemental.modalities_tidy.loc[study.supplemental.modalities_tidy.modality == 'bimodal']
bimodal = bimodal.join(study.splicing.feature_data, on='event_name')
bimodal_tfs = bimodal.loc[bimodal['gerstberger2014_rbp_target_mrna']]
bimodal_tf_events = bimodal_tfs.event_name.unique()
print bimodal_tf_events.shape

psi_bimodal_tfs = splicing_singles_no_outliers.ix[:, bimodal_tf_events].dropna(
    thresh=splicing_singles_no_outliers.shape[0]*0.5, axis=1)

row_colors = study.sample_id_to_color[psi_bimodal_tfs.index]

mask = psi_bimodal_tfs.isnull()
psi_bimodal_tfs = psi_bimodal_tfs.fillna(0.5)
xticklabels = study.splicing.feature_data.gene_name[psi_bimodal_tfs.columns]


splicing_label = 'Bimodal splicing events in RBPs'

g = sns.clustermap(psi_bimodal_tfs, mask=mask, metric='euclidean', cmap='RdYlBu_r', center=0.5, 
#                    col_cluster=False,
                   cbar_kws=dict(label='$\Psi$'), method='ward', xticklabels=xticklabels, row_colors=row_colors)
g.ax_heatmap.set(xticks=[], yticks=[], xlabel=splicing_label, ylabel='Single cells')
g.savefig('{}/bimodal_rbps_celltypes_clustergrid.pdf'.format(folder))

g = sns.clustermap(psi_bimodal_tfs.T.corr().dropna(how='all', axis=1).dropna(how='all', axis=0), 
                   metric='euclidean', cmap='PuOr', center=0, 
                   cbar_kws=dict(label='Pearson $R$'), method='ward')
g.ax_heatmap.set(xlabel='Single cells', ylabel='Single cells')
g.savefig('{}/bimodal_rbps_celltypes_samples_correlated_clustergrid.pdf'.format(folder))

g = sns.clustermap(psi_bimodal_tfs.corr().dropna(how='all', axis=1).dropna(how='all', axis=0), 
                   metric='euclidean', cmap='BrBG', center=0, 
                   cbar_kws=dict(label='Pearson $R$'), method='ward', xticklabels=xticklabels,
                  yticklabels=xticklabels)
g.ax_heatmap.set(xlabel=splicing_label, ylabel=splicing_label)
g.savefig('{}/bimodal_rbps_celltypes_events_correlated_clustergrid.pdf'.format(folder))

In [ ]:
study.plot_event('HNRNPR')

In [ ]:
study.plot_event("HNRNPC")

In [ ]:
study.plot_event('HNRNPK')

In [ ]:
psi_grouped = splicing_singles_no_outliers.groupby(study.sample_id_to_phenotype)

for phenotype, df in study.supplemental.modalities_tidy.groupby('phenotype'):
    print phenotype
    bimodal_events = df.event_name[df.modality == 'bimodal']
    psi_subset = psi_grouped.get_group(phenotype)
    print '\t', psi_subset.shape
    thresh = int(psi_subset.shape[0]*0.5)
    psi_subset = psi_subset.ix[:, bimodal_events].dropna(thresh=thresh, axis=1)
    print '\t', psi_subset.shape
    mask = psi_subset.isnull()
    psi_subset = psi_subset.fillna(0.5)
    xticklabels = study.splicing.feature_data.gene_name[psi_subset.columns]
    g = sns.clustermap(psi_subset, mask=mask, metric='euclidean', cmap='RdYlBu_r', center=0.5, 
                       cbar_kws=dict(label='$\Psi$'), method='ward', xticklabels=xticklabels)
    g.ax_heatmap.set(xticks=[], yticks=[], xlabel='Splicing events', ylabel='Single cells')
    g.savefig('{}/bimodal_{}_clustergrid.pdf'.format(folder, phenotype))
    
    
    g = sns.clustermap(psi_subset.T.corr(), metric='euclidean', cmap='PRGn', center=0, 
                       cbar_kws=dict(label='Pearson $R$'), method='ward')
    g.ax_heatmap.set(xlabel='Single cells', ylabel='Single cells', xticks=[], yticks=[])
    g.savefig('{}/bimodal_{}_samples_correlated_clustergrid.pdf'.format(folder, phenotype))

    g = sns.clustermap(psi_subset.corr(), metric='euclidean', cmap='PRGn', center=0, 
                       cbar_kws=dict(label='Pearson $R$'), method='ward', xticklabels=xticklabels,
                      yticklabels=xticklabels)
    g.ax_heatmap.set(xticks=[], yticks=[], xlabel='Splicing events', ylabel='Splicing events')
    g.savefig('{}/bimodal_{}_events_correlated_clustergrid.pdf'.format(folder, phenotype))

In [ ]:
psi_grouped = splicing_singles_no_outliers.groupby(study.sample_id_to_phenotype)

for (phenotype, modality),  df in study.supplemental.modalities_tidy.groupby(['phenotype', 'modality']):
    print phenotype, modality
#     bimodal_events = df.event_name[df.modality == 'ambiguous']
    psi_subset = psi_grouped.get_group(phenotype)
#     sns.distplot(psi_subset.var().dropna(), label=phenotype)
    
#     psi_subset = psi_subset.ix[:, psi_subset.var() > 0.1]
    
    print '\t', psi_subset.shape
#     thresh = int(psi_subset.shape[0]*0.5)
    thresh = 20
    psi_subset = psi_subset.ix[:, df.event_name].dropna(thresh=thresh, axis=1)
    print '\t', psi_subset.shape
    mask = psi_subset.isnull()
    psi_subset = psi_subset.fillna(0.5)
#     xticklabels = study.splicing.feature_data.gene_name[psi_subset.columns]
    g = sns.clustermap(psi_subset, mask=mask, metric='euclidean', cmap='RdYlBu_r', center=0.5, 
                       cbar_kws=dict(label='$\Psi$'), method='ward', )
    g.ax_heatmap.set(xticks=[], yticks=[], xlabel='Splicing events', ylabel='Single cells')

    g.savefig('{}/{}_{}_clustergrid.pdf'.format(folder, modality, phenotype))
#     g.ax_heatmap.set(xticks=[], yticks=[])
#     g.cax.set(ticks=(0, 0.5, 1))